In [1]:
import sys
from math import sqrt

In [2]:
from matplotlib import pyplot

# Data preprocessing

## Import data

In [3]:
from gcd_data_manipulation import data_aggregation, load_data, series_to_supervised

In [4]:
job_id = 6318371744
name = "[test]LSTM_workload_%i" % job_id

Load the dataset and aggregate the task values at each time step.

In [5]:
columns_to_consider = [
    "end time",
    "CPU rate",
    "canonical memory usage",
    "assigned memory usage",
    "unmapped page cache",
    "total page cache",
    "maximum memory usage",
    "disk I/O time",
    "local disk space usage",
    "maximum CPU rate",
    "maximum disk IO time",
    "cycles per instruction",
    "memory accesses per instruction",
    "CPU ratio usage",
    "memory ratio usage",
    "disk ratio usage",
    "Efficiency",  # target metric
]

readings_df = load_data(
    "data/task-usage_job-ID-%i_total.csv" % job_id, columns_to_consider
)
readings_df = data_aggregation(readings_df, aggr_type="mean")
readings_df

/opt/anaconda3/lib/python3.8/site-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,CPU rate,canonical memory usage,assigned memory usage,unmapped page cache,total page cache,maximum memory usage,disk I/O time,local disk space usage,maximum CPU rate,maximum disk IO time,cycles per instruction,memory accesses per instruction,CPU ratio usage,memory ratio usage,disk ratio usage,Efficiency
end time,,,,,,,,,,,,,,,,
900000000,0.000046,1.634317e-02,0.030983,0.000319,0.000600,0.016346,0.000003,3.179000e-09,0.000498,0.000406,15.939846,0.075165,0.000734,0.244768,0.000167,0.081890
909000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000
1195000000,0.003139,1.530000e-02,0.029750,0.000072,0.000352,0.016910,0.000096,0.000000e+00,0.031430,0.015000,1.656000,0.003803,0.050224,0.229145,0.000000,0.093123
1200000000,0.000046,1.639910e-02,0.031086,0.000319,0.000601,0.016403,0.000004,3.179000e-09,0.000496,0.000413,15.824287,0.074329,0.000735,0.245606,0.000167,0.082169
1500000000,0.000046,1.639957e-02,0.031086,0.000319,0.000601,0.016404,0.000003,3.179000e-09,0.000497,0.000355,15.769683,0.074292,0.000736,0.245613,0.000167,0.082172
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1309445000000,0.000442,1.271538e-06,0.000000,0.000002,0.000002,0.000017,NaN,0.000000e+00,0.005508,NaN,14.564359,0.066143,0.007064,0.000019,0.000000,0.002361
1309446000000,0.000444,6.935947e-05,0.000000,0.000004,0.000005,0.000463,NaN,0.000000e+00,0.005393,NaN,11.376058,0.047987,0.007107,0.001039,0.000000,0.002715
1309447000000,0.000540,9.060000e-07,0.000000,0.000000,0.000000,0.000012,NaN,0.000000e+00,0.006996,NaN,15.120595,0.068413,0.008636,0.000014,0.000000,0.002883


## Scale values

In [6]:
from gcd_data_manipulation import scale_values

values = readings_df.values
scaled, scaler = scale_values(values)
scaled

array([[2.25270301e-03, 8.38111111e-01, 8.85982271e-01, ...,
        8.38111111e-01, 3.33333333e-03, 5.34170198e-01],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [1.54174853e-01, 7.84615385e-01, 8.50729196e-01, ...,
        7.84615385e-01, 0.00000000e+00, 6.07446248e-01],
       ...,
       [2.65095776e-02, 4.64615385e-05, 0.00000000e+00, ...,
        4.64615385e-05, 0.00000000e+00, 1.88066883e-02],
       [1.43143418e-03, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 1.01390918e-03],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

## Bring data to supervised

In [7]:
from gcd_data_manipulation import extract_data_target, extract_final_dataframe

# frame as supervised learning
reframed = series_to_supervised(scaled, 1, 1)
# drop columns we don't want to predict
reframed = extract_final_dataframe(
    reframed,
    [
        [
            i
            for i in range(
                int(reframed.shape[1] / 2), (2 * int(reframed.shape[1] / 2)) - 1
            )
        ]
    ],
)


# split into train and test sets
values = reframed.values

test_X, test_y = extract_data_target(values)

/home/amorichetta/.local/lib/python3.8/site-packages/pandas/core/indexes/base.py:4111: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  result = getitem(key)


# Test model

## Load model

In [8]:
from tensorflow import keras

model_name = "results/lstm_batch72_neurons50_epochs400_do0"

model = keras.models.load_model(model_name)

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 50)                13400     
_________________________________________________________________
dense (Dense)                (None, 1)                 51        
Total params: 13,451
Trainable params: 13,451
Non-trainable params: 0
_________________________________________________________________


## Single-entry prediction function

### Generate prediction

In [9]:
from numpy import array, concatenate


def predict_model(test_X, test_y, model):
    # make a prediction
    yhat = []
    for el in test_X:
        el = el.reshape(1, 1, test_X.shape[2])
        yhat_val = model.predict(el)
        yhat.append(yhat_val[0, 0])
    test_X = test_X.reshape((test_X.shape[0], test_X.shape[2]))
    yhat = array(yhat)
    yhat = yhat.reshape(yhat.shape[0], 1)
    # invert scaling for forecast
    inv_yhat = concatenate((test_X[:, :-1], yhat), axis=1)

    inv_yhat = scaler.inverse_transform(inv_yhat)
    inv_yhat = inv_yhat[:, -1]
    # invert scaling for actual
    test_y = test_y.reshape((len(test_y), 1))
    inv_y = concatenate((test_X[:, :-1], test_y), axis=1)
    inv_y = scaler.inverse_transform(inv_y)
    inv_y = inv_y[:, -1]

    return inv_y, inv_yhat

In [10]:
y_1, ŷ_1 = predict_model(test_X, test_y, model)

### Plot results

In [26]:
%matplotlib widget

pyplot.clf()
pyplot.plot(y_1[-100:], "-", color="orange", label="Raw measurements")
pyplot.plot(ŷ_1[-100:], "--", color="blue", label="Predictions")
pyplot.xlabel("Steps", fontsize=20)
pyplot.ylabel("Efficiency", fontsize=20)
pyplot.xticks(fontsize=24)
pyplot.yticks(fontsize=24)
pyplot.legend(fontsize=14, frameon=False)
pyplot.tight_layout()
prediction_out_figure = "figures/gcd_%s_pred.png" % name

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [27]:
from math import sqrt

from sklearn.metrics import mean_squared_error

rmse = sqrt(mean_squared_error(y_1, ŷ_1))

rmse

0.04807975058161968

## Simple predict function

### Generate prediction

In [13]:
def predict_model(test_X, test_y, model):
    # make a prediction
    yhat = model.predict(test_X, batch_size=72)
    test_X = test_X.reshape((test_X.shape[0], test_X.shape[2]))
    # invert scaling for forecast
    inv_yhat = concatenate((test_X[:, :-1], yhat), axis=1)

    inv_yhat = scaler.inverse_transform(inv_yhat)
    inv_yhat = inv_yhat[:, -1]
    # invert scaling for actual
    test_y = test_y.reshape((len(test_y), 1))
    inv_y = concatenate((test_X[:, :-1], test_y), axis=1)
    inv_y = scaler.inverse_transform(inv_y)
    inv_y = inv_y[:, -1]

    return inv_y, inv_yhat

In [14]:
y_2, ŷ_2 = predict_model(test_X, test_y, model)

### Plot results

In [28]:
%matplotlib widget


pyplot.clf()
pyplot.plot(y_2[-100:], "-", color="orange", label="Raw measurements")
pyplot.plot(ŷ_2[-100:], "--", color="blue", label="Predictions")
pyplot.xlabel("Steps", fontsize=20)
pyplot.ylabel("Efficiency", fontsize=20)
pyplot.xticks(fontsize=24)
pyplot.yticks(fontsize=24)
pyplot.legend(fontsize=14, frameon=False)
pyplot.tight_layout()
prediction_out_figure = "figures/gcd_%s_pred.png" % name

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [29]:
rmse = sqrt(mean_squared_error(y_2, ŷ_2))

rmse

0.04807975057008824

## Predict with update

### Generate predict model function

In [17]:
def predict_model(test_X, test_y, model):
    # make a prediction
    yhat = []
    for i, el in enumerate(test_X):
        if (i % 72 == 0) and i != 0:
            print(i)
            update_model_set = test_X[:i, :, :]
            update_model_y = test_y[:i]
            model.fit(update_model_set, update_model_y, epochs=5, shuffle=False)
        el = el.reshape(1, 1, test_X.shape[2])
        yhat_val = model.predict(el)
        yhat.append(yhat_val[0, 0])

    test_X = test_X.reshape((test_X.shape[0], test_X.shape[2]))
    yhat = array(yhat)
    yhat = yhat.reshape(yhat.shape[0], 1)
    # invert scaling for forecast
    inv_yhat = concatenate((test_X[:, :-1], yhat), axis=1)

    inv_yhat = scaler.inverse_transform(inv_yhat)
    inv_yhat = inv_yhat[:, -1]
    # invert scaling for actual
    test_y = test_y.reshape((len(test_y), 1))
    inv_y = concatenate((test_X[:, :-1], test_y), axis=1)
    inv_y = scaler.inverse_transform(inv_y)
    inv_y = inv_y[:, -1]

    return inv_y, inv_yhat

### Instantiate a copy of the original model - to not pollute the original one

In [18]:
model_name = "results/lstm_batch72_neurons50_epochs400_do0"

model_update = keras.models.load_model(model_name)

### Generate predicrion

In [ ]:
y_3, ŷ_3 = predict_model(test_X, test_y, model_update)

72
Epoch 1/5
3/3 [==============================] - 0s 4ms/step - loss: 0.3795
Epoch 2/5
3/3 [==============================] - 0s 3ms/step - loss: 0.3616
Epoch 3/5
3/3 [==============================] - 0s 4ms/step - loss: 0.3435
Epoch 4/5
3/3 [==============================] - 0s 4ms/step - loss: 0.3250
Epoch 5/5
3/3 [==============================] - 0s 4ms/step - loss: 0.3061
144
Epoch 1/5
5/5 [==============================] - 0s 4ms/step - loss: 0.2509
Epoch 2/5
5/5 [==============================] - 0s 4ms/step - loss: 0.2179
Epoch 3/5
5/5 [==============================] - 0s 4ms/step - loss: 0.1838
Epoch 4/5
5/5 [==============================] - 0s 5ms/step - loss: 0.1484
Epoch 5/5
5/5 [==============================] - 0s 5ms/step - loss: 0.1127
216
Epoch 1/5
7/7 [==============================] - 0s 4ms/step - loss: 0.0743
Epoch 2/5
7/7 [==============================] - 0s 4ms/step - loss: 0.0740
Epoch 3/5
7/7 [==============================] - 0s 5ms/step - loss: 0.0691
E

### Plot results

In [1]:
%matplotlib widget


pyplot.clf()
pyplot.plot(y_3[-100:], "-", color="orange", label="Raw measurements")
pyplot.plot(ŷ_3[-100:], "--", color="blue", label="Predictions")
pyplot.xlabel("Steps", fontsize=20)
pyplot.ylabel("Efficiency", fontsize=20)
pyplot.xticks(fontsize=24)
pyplot.yticks(fontsize=24)
pyplot.legend(fontsize=14, frameon=False)
pyplot.tight_layout()
prediction_out_figure = "figures/gcd_%s_pred.png" % name

NameError: name 'pyplot' is not defined

In [23]:
rmse = sqrt(mean_squared_error(y_3, ŷ_3))

rmse

0.03103470455499695